# Dimension-Dependence of MFGS

<span style="font-size:1.2em;">! Run this in **Julia** !</span> 

1. You'll need to install the Julia add on for Jupyter Notebooks (IJulia). Follow guide here: https://datatofish.com/add-julia-to-jupyter/
2. Go to *Kernel* -> *Change Kernel* -> *Julia*

## Packages

In [1]:
include("RCLib.jl") # Include the RC library with all the useful functions
using .RCLib

using LinearAlgebra # For IIL matrix multiplication
using Kronecker
using OrdinaryDiffEq
using DifferentialEquations

## Plotting Packages ##
using CSV # Allows data to be converted to .CSV to use with Python for plotting
using DataFrames # Another useful package for exporting data
using Plots

## Performance Packages ##
using BenchmarkTools

## Temp Lists

In [2]:
T_list = exp10.(range(-3, 4, length=100));

In [3]:
T = [i for i in T_list];
sz3DMFGS_cl = [pred(i) for i in T_list];

## Classical

In [4]:
df_cl = DataFrame(hcat(T, sz3DMFGS_cl), :auto);

In [5]:
# CSV.write("C://Users//crh222//QuantumSpinDynamics//MFGS_data_cl_pred.csv",  df_cl, header = ["T", "sz3D_cl"]);

## Quantum

### Variables

#### Lorentzian Spectral Density

In [6]:
prma = [1.4 0 10];
prmb = [7 0 50]; # SAFE parameter set
prmc = [0.14 0 0.1];
prmd = [0.014 0 0.001];
prm = prmd;

In [7]:
ω0x, Γx, αx = prm*ωL;
ω0y, Γy, αy = prm*ωL;
ω0z, Γz, αz = prm*ωL;

#### RC-Specific

In [8]:
Ωx = ω0x;
λx = ωL*sqrt(αx/Ωx);
δx = Γx/(2π*Ωx);
nx = 5;

In [9]:
Ωy = ω0y;
λy = ωL*sqrt(αy/Ωy);
δy = Γy/(2π*Ωy);
ny = 5;

In [10]:
Ωz = ω0z;
λz = ωL*sqrt(αz/Ωz);
δz = Γz/(2π*Ωz);
nz = 5;

### Gibbs State

In [11]:
ρSpG(T) = gibbs(HSpG, T);

In [12]:
ρSpG(10)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [13]:
tr(ρSpG(10))

1.0

## 1D Mean-Force Gibbs State


In [14]:
Htot1D = HRC1D(nx, λx, Ωx); # System Hamiltonian in original basis

In [15]:
P1D = eigen(Htot1D).vectors; # Find the projectors onto the system eigenbasis

In [16]:
Htot1DT = adjoint(P1D)*Htot1D*P1D; # System Hamiltonian in its eigenbasis

In [17]:
ρtot1D(T) = P1D*gibbs(Htot1DT, T)*adjoint(P1D); # Total Gibbs state in original basis

In [18]:
ρMFGS1D(T) = ptrace(ρtot1D(T), nx); # Trace out the bath

In [19]:
ρMFGS1D(10) # Resulting MFGS

2×2 Matrix{Float64}:
  0.217516     -6.30351e-17
 -6.30351e-17   0.782484

In [20]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [21]:
tr(ρMFGS1D(1))

0.9999999999999999

### Ground-State Test

In [22]:
GS = eigvecs(Htot1D)[:,1];

In [23]:
exGS1D = -adjoint(GS)*(sz0 ⊗ Matrix(I, nx, nx))*GS

0.39888830755405463

## 2D Mean-Force Gibbs State

In [24]:
Htot2D = HRC2D(nx, nz, λx, λz, Ωx, Ωz); # System Hamiltonian in original basis

In [25]:
P2D = eigen(Htot2D).vectors; # Find the projectors onto the system eigenbasis

In [26]:
Htot2DT = adjoint(P2D)*Htot2D*P2D; # System Hamiltonian in its eigenbasis

In [27]:
ρtot2D(T) = P2D*gibbs(Htot2DT, T)*adjoint(P2D); # Total Gibbs state in original basis

In [28]:
ρMFGS2D(T) = ptrace(ρtot2D(T), nx*nz); # Trace out the bath

In [29]:
ρMFGS2D(10) # Resulting MFGS

2×2 Matrix{Float64}:
 0.217057     5.56496e-15
 5.56496e-15  0.782943

In [30]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [31]:
tr(ρMFGS2D(1))

0.9999999999999999

### Ground-State Test

In [32]:
GS = eigvecs(Htot2D)[:,1];

In [33]:
exGS2D = -adjoint(GS)*(sz0 ⊗ Matrix(I, nx*nz, nx*nz))*GS;

## 3D Mean-Force Gibbs State

In [34]:
Htot3D = HRC3D(nx, ny, nz, λx, λy, λz, Ωx, Ωy, Ωz); # System Hamiltonian in original basis

In [35]:
P3D = eigen(Htot3D).vectors; # Find the projectors onto the system eigenbasis

In [36]:
Htot3DT = adjoint(P3D)*Htot3D*P3D; # System Hamiltonian in its eigenbasis

In [37]:
ρtot3D(T) = P3D*gibbs(Htot3DT, T)*adjoint(P3D); # Total Gibbs state in original basis

In [38]:
ρMFGS3D(T) = ptrace(ρtot3D(T), nx*ny*nz); # Trace out the bath

In [39]:
ρMFGS3D(10) # Resulting MFGS

2×2 Matrix{ComplexF64}:
    0.225749+7.43377e-19im  7.72885e-15+6.42915e-15im
 7.72885e-15-6.42915e-15im     0.774251-1.03245e-18im

In [40]:
ρSpG(10) # Compare to the Gibbs state (should be the same when λ = 0)

2×2 Matrix{Float64}:
 0.207653  0.0
 0.0       0.792347

In [41]:
tr(ρMFGS3D(1))

0.9999999999999996 + 2.1164020681477838e-18im

### Ground-State Test

In [42]:
GS = eigvecs(Htot3D)[:,1];

In [43]:
exGS3D = -adjoint(GS)*(sz0⊗ Matrix(I, nx*ny*nz, nx*ny*nz))*GS

0.42943898114872403 + 0.0im

# Plots

In [44]:
s0 = sz0;

In [45]:
expG(T) = tr(ρSpG(T)*s0); # Gibbs Sz expectation value
exp1DMFGS(T) = tr(ρMFGS1D(T)*s0); # 1D MFGS Sz expectation value
exp2DMFGS(T) = tr(ρMFGS2D(T)*s0); # 2D MFGS Sz expectation value
exp3DMFGS(T) = tr(ρMFGS3D(T)*s0); # 3D MFGS Sz expectation value

In [46]:
sG = [real_if_close(expG(i)) for i in T_list];
s1DMFGS = [real_if_close(exp1DMFGS(i)) for i in T_list];
s2DMFGS = [real_if_close(exp2DMFGS(i)) for i in T_list];
s3DMFGS = [real_if_close(exp3DMFGS(i)) for i in T_list];
GS1D = [real_if_close(exGS1D) for i in T_list];
GS2D = [real_if_close(exGS2D) for i in T_list];
GS3D = [real_if_close(exGS3D) for i in T_list];
df_qu = DataFrame(hcat(T, sG, s1DMFGS, s2DMFGS, s3DMFGS, GS1D, GS2D, GS3D), :auto);

In [47]:
CSV.write("C://Users//crh222//QuantumSpinDynamics//MFGS_data_qu.csv",  df_qu, header = ["T", "sG", "s1DMFGS", "s2DMFGS", "s3DMFGS", "GS1D", "GS2D", "GS3D"]);